In [2]:
base_url = 'https://portal.dnb.de/opac/showPreviousResultSite?currentResultId=tit+all+%22security%22%26any%26dnb.hss%26onlinefree'

import requests
resp = requests.get(base_url)


In [3]:
from bs4 import BeautifulSoup
from p_tqdm import p_map

html = resp.text
soup = BeautifulSoup(html, "html.parser")
import tqdm

overall_number = int(soup.find_all("span", {"class": "amount"})[0].text.split(' ')[-1])

def poolFunction(i):
    details = (base_url.replace('showPreviousResultSite','showFullRecord')+'&currentPosition='+str(i))
    item = requests.get(details).text
    soup = BeautifulSoup(item, "html.parser")
    js = {}
    for tr in soup.find_all("tr"):
        try:
            href = tr.find_all('td')[1].find('a')
            if href:
                js[tr.find('strong').text.strip()] = href['href']
            else:
                js[tr.find('strong').text.strip()] = tr.find_all('td')[1].text.strip()
        except:
            pass
    return js

#print(poolFunction(1))
r = p_map(poolFunction, list(range(0, overall_number)))

100%|██████████| 438/438 [01:34<00:00,  4.66it/s]


In [6]:
def getPDF(url):
    if '.pdf' in url:
        print(url)
    else:
        tuprints = requests.get(url).text
        soup = BeautifulSoup(tuprints, "html.parser")
        baseurl = '/'.join(url.split('/')[:3])
        hrefs = soup.find_all('a')
        pdfs = {}
        for href in hrefs:
            if href.has_attr('href') and 'pdf' in href['href']:
                pdfs[href['href']]=True
            if href.has_attr('class') and href['class'] == 'downloadTextLink':
                pdfs[href['href']]=True
        if not len(pdfs):
            print('WARNING',url)
        for pdf in pdfs.keys():
            if pdf.startswith('http'):
                print(pdf)
            else:
                print(baseurl+pdf)

In [8]:
for element in r:
    if 'URL' not in element or 'http' not in element['URL']:
        link = (element['Online-Zugriff'])
    else:
        link = (element['URL'])
    if 'd-nb.info' not in link:
        print(link)
        try:
            getPDF(link)
        except:
            pass
    #print(element['URL'])

http://nbn-resolving.de/urn:nbn:de:bsz:289-oparu-34231-0
http://nbn-resolving.de/xmlui/bitstream/handle/123456789/34231/Masterarbeit.pdf?sequence=5&isAllowed=y
http://hdl.handle.net/11420/8304
http://hdl.handle.net/bitstream/11420/8304/3/Dissertation%20Tavseef%20Mairaj%20Shah%20-%20TORE%20Version.pdf
https://publikationsserver.tu-braunschweig.de/receive/dbbs_mods_00069353
https://publikationsserver.tu-braunschweig.de/servlets/MCRFileNodeServlet/dbbs_derivate_00048021/Diss_Engelmann_Anna.pdf
https://refubium.fu-berlin.de/handle/fub188/31783
https://refubium.fu-berlin.de/bitstream/handle/fub188/31783/Mukherjee_dissertation_urban_water_security_Kolkata.pdf?sequence=4&isAllowed=y
https://refubium.fu-berlin.de/bitstream/handle/fub188/31783/Mukherjee_dissertation_urban_water_security_Kolkata.pdf?sequence=4&isAllowed=y&save=y
http://nbn-resolving.de/urn:nbn:de:bvb:91-diss-20210817-1584808-1-5
http://nbn-resolving.de/doc/1584808/1584808.pdf
http://nbn-resolving.de/download/1584808/1584808.pdf


https://tuprints.ulb.tu-darmstadt.de
36
https://tuprints.ulb.tu-darmstadt.de/18901/1/Dissertation_Margareta%20Heidt_final.pdf
https://tuprints.ulb.tu-darmstadt.de/18901/1.haslightboxThumbnailVersion/Dissertation_Margareta%20Heidt_final.pdf
